# Análise do Perfil Nutricional Por Região

In [22]:
%defaultDatasource jdbc:h2:mem:db

## Ingestão dos Dados do FooDB

<a href='https://foodb.ca/'>`https://foodb.ca/`</a>

O FooDB apresenta informações sobre a composição química e nutricional dos alimentos. Para a nossa análise, vamos considerar apenas alguns nutrientes.

In [2]:
DROP VIEW IF EXISTS RecipeProfile;
DROP TABLE IF EXISTS Nutrient;
DROP TABLE IF EXISTS Food;
DROP TABLE IF EXISTS Nutrient_On_Food;

CREATE TABLE Nutrient (
    id INT,
    public_id VARCHAR(15),
    Nutrient_Name VARCHAR(50),
    PRIMARY KEY(id)
) AS SELECT id, public_id, name as Nutrient_Name
    FROM CSVREAD('../data/raw/foodb/Nutrient.csv');

CREATE TABLE Food (
    Id INT,
    Food_Name VARCHAR(50),
    Description VARCHAR(4000),
    Food_Group VARCHAR(50),
    Food_Subgroup VARCHAR(50),
    Public_Id VARCHAR(50),
    PRIMARY KEY(Id)
) AS SELECT Id, name as Food_Name, Description, Food_Group, Food_Subgroup, Public_Id
    FROM CSVREAD('../data/raw/foodb/Food.csv');
    
CREATE TABLE Nutrient_On_Food (
    Id INT,
    Nutrient_Id INT,
    Food_Id INT,
    Orig_Content FLOAT,
    Unit VARCHAR(10),
    Quantity_Ref FLOAT,
    PRIMARY KEY(Id),
    FOREIGN KEY(Nutrient_Id)
        REFERENCES Nutrient(id)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY(Food_Id)
        REFERENCES Food(Id)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION
) AS SELECT C.Id, C.Source_Id AS Nutrient_Id, C.Food_Id, C.Orig_Content, C.Unit, C.Quantity_Ref
    FROM CSVREAD('../data/interim/ProcessedContent.csv') C, Food F
    WHERE Food_Id=F.Id;

## Ingestão dos Dados do CulinaryDB

<a href='https://cosylab.iiitd.edu.in/culinarydb/'>`https://cosylab.iiitd.edu.in/culinarydb`</a>
    

In [3]:
DROP VIEW IF EXISTS RecipeProfile;
DROP TABLE IF EXISTS Recipes;
DROP TABLE IF EXISTS Ingredients;
DROP TABLE IF EXISTS Units;

CREATE TABLE Recipes (
    Recipe_ID INT,
    Title VARCHAR(250),
    Region VARCHAR(50),
    PRIMARY KEY(Recipe_ID)
) AS SELECT Recipe_ID, Title, Region
    FROM CSVREAD('../data/interim/Recipes.csv');

CREATE TABLE Units (
    Unit_Name VARCHAR(15),
    Grams FLOAT,
    PRIMARY KEY(Unit_Name)
) AS SELECT Unit_Name, Grams
    FROM CSVREAD('../data/interim/Units-Conversion.csv');

CREATE TABLE Ingredients (
    Recipe_ID INT,
    Food_ID INT,
    Food_Name VARCHAR(50),
    Quantity FLOAT,
    Unit VARCHAR(15),
    PRIMARY KEY(Recipe_ID, Food_ID),
    FOREIGN KEY(Recipe_ID)
        REFERENCES Recipes(Recipe_ID)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY(Food_ID)
        REFERENCES Food(Id)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY(Unit)
        REFERENCES Units(Unit_name)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION
) AS SELECT Recipe_ID, Food_ID, Aliased_Ingredient_Name as Food_Name, Quantity, Unit
    FROM CSVREAD('../data/interim/IngredientOnFood2.csv');

# Pergunta e Análise 2:

- Quais os alimentos e receitas que mais contribuem para a ingestão de açúcares, gorduras e sódio para cada país/região?



In [ ]:
DROP VIEW IF EXISTS RecipeProfile;

CREATE VIEW RecipeProfile AS
SELECT R.Recipe_ID
    FROM Recipes R, Ingredients I, Units U, Nutrient_On_Food NF, Nutrient N, Food F
    WHERE F.Food_Name = I.Food_Name AND R.Recipe_ID = I.Recipe_ID AND U.Unit_Name = I.Unit;

SELECT COUNT(*) FROM RecipeProfile;

0

In [23]:
SELECT COUNT(*) FROM Food F, Ingredients I
WHERE I.Food_Name LIKE 'kiwi';

67456

In [24]:
SELECT COUNT(*) FROM Ingredients
WHERE Food_Name LIKE 'Kiwi';

0